<a href="https://colab.research.google.com/github/gurukite/Ngulik-Pendidikan/blob/main/01.%20Ngulikin%20-%20Prediksi%20Kebutuhan%20Guru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ===========================
# Prediksi Kebutuhan Guru
# ===========================

In [5]:
# 1. Import Library
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
import os
from io import StringIO

In [8]:
# ===========================
# 2. Load Data (CSV eksternal atau contoh internal)
# ===========================
csv_file = 'dataset_guru.csv'

try:
    df = pd.read_csv(csv_file)
    print(f"Dataset '{csv_file}' berhasil dimuat.")
except FileNotFoundError:
    print(f"Dataset '{csv_file}' tidak ditemukan. Menggunakan contoh dataset internal.")
    csv_data = StringIO("""
tahun,sekolah_id,sekolah_nama,kecamatan,desa,jenjang,jumlah_siswa,jumlah_guru_ada,mata_pelajaran,guru_ada_per_mapel,rasio_siswa_per_guru_ideal
2022,SDN001,SDN 1 Kecamatan A,Kecamatan A,Desa X,SD,120,10,Matematika,2,30
2022,SDN001,SDN 1 Kecamatan A,Kecamatan A,Desa X,SD,120,10,IPA,1,30
2022,SDN002,SDN 2 Kecamatan A,Kecamatan A,Desa Y,SD,150,12,Matematika,3,30
2022,SDN002,SDN 2 Kecamatan A,Kecamatan A,Desa Y,SD,150,12,IPA,2,30
2022,SMP001,SMP 1 Kecamatan B,Kecamatan B,Desa Z,SMP,200,8,Matematika,2,25
2022,SMP001,SMP 1 Kecamatan B,Kecamatan B,Desa Z,SMP,200,8,Bahasa Inggris,1,25
2022,SMP002,SMP 2 Kecamatan B,Kecamatan B,Desa W,SMP,180,9,Matematika,2,25
2022,SMP002,SMP 2 Kecamatan B,Kecamatan B,Desa W,SMP,180,9,Bahasa Inggris,2,25
""")
    df = pd.read_csv(csv_data)
    print(df.head())


Dataset 'dataset_guru.csv' tidak ditemukan. Menggunakan contoh dataset internal.
   tahun sekolah_id       sekolah_nama    kecamatan    desa jenjang  \
0   2022     SDN001  SDN 1 Kecamatan A  Kecamatan A  Desa X      SD   
1   2022     SDN001  SDN 1 Kecamatan A  Kecamatan A  Desa X      SD   
2   2022     SDN002  SDN 2 Kecamatan A  Kecamatan A  Desa Y      SD   
3   2022     SDN002  SDN 2 Kecamatan A  Kecamatan A  Desa Y      SD   
4   2022     SMP001  SMP 1 Kecamatan B  Kecamatan B  Desa Z     SMP   

   jumlah_siswa  jumlah_guru_ada mata_pelajaran  guru_ada_per_mapel  \
0           120               10     Matematika                   2   
1           120               10            IPA                   1   
2           150               12     Matematika                   3   
3           150               12            IPA                   2   
4           200                8     Matematika                   2   

   rasio_siswa_per_guru_ideal  
0                          30  
1

In [11]:
# ===========================
# 3. Data Cleaning & Validasi
# ===========================
df.drop_duplicates(inplace=True)
numeric_cols = ['jumlah_siswa', 'jumlah_guru_ada', 'guru_ada_per_mapel', 'rasio_siswa_per_guru_ideal']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.fillna(0, inplace=True)

In [10]:
# ===========================
# 4. Hitung Rasio Siswa/Guru
# ===========================
df['rasio_siswa_per_guru'] = df['jumlah_siswa'] / df['guru_ada_per_mapel']

In [12]:
# ===========================
# 5a. Forecast Pertumbuhan Siswa - 5% Growth
# ===========================
df['proyeksi_siswa'] = (df['jumlah_siswa'] * 1.05).round().astype(int)

In [13]:
# ===========================
# 5b. Forecast Pertumbuhan Siswa - Prophet per Sekolah
# ===========================
def forecast_siswa_prophet(school_id, periods=1):
    df_school = df[df['sekolah_id'] == school_id][['tahun', 'jumlah_siswa']].copy()
    if df_school.empty:
        return 0
    df_school.rename(columns={'tahun': 'ds', 'jumlah_siswa': 'y'}, inplace=True)
    model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
    try:
        model.fit(df_school)
        future = model.make_future_dataframe(periods=periods, freq='Y')
        forecast = model.predict(future)
        return forecast[['ds', 'yhat']].tail(periods)['yhat'].values[0]
    except Exception as e:
        print(f"Forecast gagal untuk sekolah {school_id}: {e}")
        # fallback: growth 5%
        current_students = df_school['y'].iloc[-1] if not df_school.empty else 0
        return current_students * 1.05

df['proyeksi_siswa_prophet'] = df['sekolah_id'].apply(lambda x: forecast_siswa_prophet(x, periods=1))
df['proyeksi_siswa_prophet'] = df['proyeksi_siswa_prophet'].round().astype(int)

INFO:prophet:n_changepoints greater than number of observations. Using 0.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1378: RuntimeWarning: Mean of empty slice
  k = np.nanmean(self.params['k'])
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1379: RuntimeWarning: Mean of empty slice
  m = np.nanmean(self.params['m'])
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,
INFO:prophet:n_changepoints greater than number of observations. Using 0.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
/usr/local/lib/python3.12/dist-packages/proph

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [14]:
# ===========================
# 6. Hitung Kebutuhan Guru Masa Depan
# ===========================
df['jumlah_guru_dibutuhkan'] = np.ceil(df['proyeksi_siswa'] / df['rasio_siswa_per_guru_ideal'])
df['kekurangan_guru'] = df['jumlah_guru_dibutuhkan'] - df['guru_ada_per_mapel']
df['kekurangan_guru'] = df['kekurangan_guru'].apply(lambda x: max(x, 0))

df['jumlah_guru_dibutuhkan_prophet'] = np.ceil(df['proyeksi_siswa_prophet'] / df['rasio_siswa_per_guru_ideal'])
df['kekurangan_guru_prophet'] = df['jumlah_guru_dibutuhkan_prophet'] - df['guru_ada_per_mapel']
df['kekurangan_guru_prophet'] = df['kekurangan_guru_prophet'].apply(lambda x: max(x,0))

def prioritas(kekurangan):
    if kekurangan >= 3:
        return 'Tinggi'
    elif kekurangan == 2:
        return 'Sedang'
    elif kekurangan == 1:
        return 'Rendah'
    else:
        return 'Cukup'

df['prioritas'] = df['kekurangan_guru'].apply(prioritas)
df['rekomendasi_penempatan'] = df['prioritas'].apply(
    lambda x: 'Rekrut baru' if x in ['Tinggi','Sedang'] else 'Tidak perlu'
)
df['prioritas_prophet'] = df['kekurangan_guru_prophet'].apply(prioritas)
df['rekomendasi_penempatan_prophet'] = df['prioritas_prophet'].apply(
    lambda x: 'Rekrut baru' if x in ['Tinggi','Sedang'] else 'Tidak perlu'
)


In [15]:
# ===========================
# 7. Agregasi Per Jenjang & Kecamatan
# ===========================
agg_kecamatan = df.groupby(['kecamatan','jenjang']).agg({
    'jumlah_siswa':'sum',
    'jumlah_guru_ada':'sum',
    'jumlah_guru_dibutuhkan':'sum',
    'kekurangan_guru':'sum'
}).reset_index()

agg_kecamatan_prophet = df.groupby(['kecamatan','jenjang']).agg({
    'jumlah_siswa':'sum',
    'jumlah_guru_ada':'sum',
    'jumlah_guru_dibutuhkan_prophet':'sum',
    'kekurangan_guru_prophet':'sum'
}).reset_index()

In [16]:
# ===========================
# 8. Export Hasil
# ===========================
os.makedirs('output', exist_ok=True)
df.to_csv('output/hasil_prediksi_guru_per_sekolah.csv', index=False)
agg_kecamatan.to_csv('output/hasil_prediksi_guru_per_kecamatan.csv', index=False)
df.to_csv('output/hasil_prediksi_guru_prophet_per_sekolah.csv', index=False)
agg_kecamatan_prophet.to_csv('output/hasil_prediksi_guru_prophet_per_kecamatan.csv', index=False)
print("Prediksi kebutuhan guru berhasil dihitung dan diekspor di folder 'output/'.")


Prediksi kebutuhan guru berhasil dihitung dan diekspor di folder 'output/'.


In [17]:
# ===========================
# 9. Visualisasi Distribusi Guru & Simpan Chart
# ===========================
os.makedirs('charts', exist_ok=True)

# a) Kekurangan Guru per Kecamatan (5% growth)
plt.figure(figsize=(10,6))
sns.barplot(data=agg_kecamatan, x='kecamatan', y='kekurangan_guru', hue='jenjang')
plt.title('Kekurangan Guru per Kecamatan dan Jenjang')
plt.ylabel('Kekurangan Guru')
plt.xlabel('Kecamatan')
plt.xticks(rotation=45)
plt.legend(title='Jenjang')
plt.tight_layout()
plt.savefig('charts/kekurangan_guru_per_kecamatan.png')
plt.close()

# b) Kekurangan Guru per Kecamatan (Prophet)
plt.figure(figsize=(10,6))
sns.barplot(data=agg_kecamatan_prophet, x='kecamatan', y='kekurangan_guru_prophet', hue='jenjang')
plt.title('Kekurangan Guru per Kecamatan dan Jenjang (Prophet Forecast)')
plt.ylabel('Kekurangan Guru')
plt.xlabel('Kecamatan')
plt.xticks(rotation=45)
plt.legend(title='Jenjang')
plt.tight_layout()
plt.savefig('charts/kekurangan_guru_prophet_per_kecamatan.png')
plt.close()

# c) Distribusi Guru per Mata Pelajaran
plt.figure(figsize=(12,6))
sns.barplot(data=df, x='mata_pelajaran', y='guru_ada_per_mapel', hue='jenjang')
plt.title('Distribusi Guru per Mata Pelajaran dan Jenjang')
plt.ylabel('Jumlah Guru')
plt.xlabel('Mata Pelajaran')
plt.xticks(rotation=45)
plt.legend(title='Jenjang')
plt.tight_layout()
plt.savefig('charts/distribusi_guru_per_mapel.png')
plt.close()

# d) Heatmap Kekurangan Guru per Sekolah (5% growth)
pivot = df.pivot_table(index='sekolah_nama', columns='mata_pelajaran', values='kekurangan_guru', fill_value=0)
plt.figure(figsize=(12,8))
sns.heatmap(pivot, annot=True, cmap='Reds')
plt.title('Heatmap Kekurangan Guru per Sekolah dan Mata Pelajaran')
plt.ylabel('Sekolah')
plt.xlabel('Mata Pelajaran')
plt.tight_layout()
plt.savefig('charts/heatmap_kekurangan_guru.png')
plt.close()

# e) Heatmap Kekurangan Guru per Sekolah (Prophet)
pivot_prophet = df.pivot_table(index='sekolah_nama', columns='mata_pelajaran', values='kekurangan_guru_prophet', fill_value=0)
plt.figure(figsize=(12,8))
sns.heatmap(pivot_prophet, annot=True, cmap='Reds')
plt.title('Heatmap Kekurangan Guru per Sekolah dan Mata Pelajaran (Prophet Forecast)')
plt.ylabel('Sekolah')
plt.xlabel('Mata Pelajaran')
plt.tight_layout()
plt.savefig('charts/heatmap_kekurangan_guru_prophet.png')
plt.close()

print("Semua chart berhasil disimpan di folder 'charts/'")

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1200x800 with 0 Axes>